### Análise exploratória health lifestyle dataset

In [49]:
import pandas as pd
import numpy as np
from pathlib import Path

In [50]:
# File path made easier 
caminho_arquivo = Path('clean_new_data_obesity.csv').resolve().parents[1]
print('caminho do diretório',caminho_arquivo)

caminho do diretório D:\python\bellabeat-analysis


In [51]:
df = pd.read_csv(f'{caminho_arquivo}/database/health-lifestyle-data/health_activity_data_treated.csv')

## Análise exploratória e estatística inicial

In [52]:
df.head()

,ID,Age,Age_Group,Gender,Gender_bool,Height_cm,Weight_kg,BMI,BMI_Category,Daily_Steps,...,Heart_Rate,Blood_Pressure,Exercise_Hours_per_Week,Smoker,Smoker_bool,Alcohol_Consumption_per_Week,Diabetic,Diabetic_bool,Heart_Disease,Heart_Disease_bool
0,1,56,50 - 59,Male,0,164,81,30.72,Obese,5134,...,102,137/72,8.1,No,0,7,No,0,No,0
1,2,69,60 - 69,Male,0,156,82,20.86,Normal,12803,...,103,129/65,3.7,No,0,7,No,0,No,0
2,3,46,40 - 49,Female,1,158,65,30.93,Obese,16408,...,74,127/68,3.2,Yes,1,0,No,0,No,0
3,4,32,29 - 39,Male,0,197,87,31.19,Obese,18420,...,116,125/86,8.5,No,0,5,No,0,No,0
4,5,60,60 - 69,Male,0,157,63,29.37,Overweight,17351,...,111,100/64,8.5,Yes,1,8,No,0,No,0


In [53]:
df[['Age', 'Daily_Steps', 'Calories_Intake', 'Hours_of_Sleep', 'Heart_Rate', 'Exercise_Hours_per_Week']].describe().round(1)

,Age,Daily_Steps,Calories_Intake,Hours_of_Sleep,Heart_Rate,Exercise_Hours_per_Week
count,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
mean,49.9,10717.0,2327.1,6.9,84.7,5.0
std,18.1,5444.7,657.8,1.8,20.1,2.8
min,18.0,1016.0,1201.0,4.0,50.0,0.0
25%,35.0,6028.2,1745.8,5.3,67.0,2.7
50%,50.0,10898.0,2328.5,6.9,84.0,5.1
75%,66.0,15253.2,2880.0,8.4,103.0,7.6
max,79.0,19931.0,3498.0,10.0,119.0,10.0


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            1000 non-null   int64  
 1   Age                           1000 non-null   int64  
 2   Age_Group                     1000 non-null   object 
 3   Gender                        1000 non-null   object 
 4   Gender_bool                   1000 non-null   int64  
 5   Height_cm                     1000 non-null   int64  
 6   Weight_kg                     1000 non-null   int64  
 7   BMI                           1000 non-null   float64
 8   BMI_Category                  1000 non-null   object 
 9   Daily_Steps                   1000 non-null   int64  
 10  Calories_Intake               1000 non-null   int64  
 11  Hours_of_Sleep                1000 non-null   float64
 12  Heart_Rate                    1000 non-null   int64  
 13  Bloo

### Como o Bellabeat tem público feminino, fazer recorte de gênero no dataset para encontrar padrões de exercício e estilo de vida em mulheres

In [55]:
df['Gender'].value_counts()

Gender
Male      523
Female    477
Name: count, dtype: int64

In [56]:
df_fem = df.loc[df['Gender'] == 'Female']

#### Análise de daily steps

In [25]:
daily_steps_mean = df_fem['Daily_Steps'].mean().round(1)

#### Contagem de BMI category das pessoas com média de passos abaixo da média

In [69]:
few_steps_pop = df_fem.loc[df_fem['Daily_Steps']< daily_steps_mean, 'BMI_Category'].value_counts()
few_steps_pop

BMI_Category
Normal        88
Obese         83
Overweight    72
Name: count, dtype: int64

#### Contagem de BMI category das pessoas com média de passos acima da média

In [27]:
more_steps_pop = df_fem.loc[df_fem['Daily_Steps']> daily_steps_mean, 'BMI_Category'].value_counts()
more_steps_pop

BMI_Category
Normal        90
Obese         85
Overweight    59
Name: count, dtype: int64

##### Para essa amostra, passos e IMC não estão relacionados

### Exercício por semana por categoria de IMC

In [59]:
pd.pivot_table(df_fem, values = 'Exercise_Hours_per_Week', columns= 'BMI_Category', aggfunc= 'mean').round(1)

BMI_Category,Normal,Obese,Overweight
Exercise_Hours_per_Week,5.1,4.9,4.8


### Pivot table on BMI categories, diabetes e heart disease

In [30]:
# tabela para analisar como doença cardiaca e diabetes afeta qntde de passos diário
pd.pivot_table(df_fem, values= 'Daily_Steps', index=('Heart_Disease', 'Diabetic'), columns= 'BMI_Category', aggfunc= 'mean').round(1)

BMI_Category             Normal    Obese  Overweight
Heart_Disease Diabetic                              
No            No        10834.0  10546.2     10137.7
              Yes       10534.7  10516.0     10553.6
Yes           No         9788.9   7783.0      8251.2
              Yes       13205.0  16609.7         NaN

In [31]:
# tabela para analisar como doença cardiaca e diabetes afeta tempo de exercício
pd.pivot_table(df_fem, values= 'Exercise_Hours_per_Week', index=('Heart_Disease', 'Diabetic'), columns= 'BMI_Category', aggfunc= 'mean').round(1)

BMI_Category            Normal  Obese  Overweight
Heart_Disease Diabetic                           
No            No           5.3    4.9         4.6
              Yes          4.3    5.4         5.4
Yes           No           5.0    4.1         5.5
              Yes          6.5    6.6         NaN

Pessoas diabeticas de todas as categorias de BMI tem média diária de passo maior quando comparadas com a mesma <br>categoria mas sem diabete. 
Doença cardíaca não apresenta o mesmo padrão (pessoas com doenças cardíacas sem diabetes <br> andam menos do que pessoas sem doença cardíaca sem diabete)
O grupo doença cardíaca [Sim] e diabete [não]<br> é o que tem menor média de passos

### Análise de habitos por idade

In [73]:
pd.pivot_table(df_fem, values= ('Daily_Steps', 'Calories_Intake', 'Hours_of_Sleep', 'Exercise_Hours_per_Week', 'Alcohol_Consumption_per_Week'), 
               columns= ('Age_Group'), 
               aggfunc={'Daily_Steps':'median', 'Calories_Intake':'median', 'Hours_of_Sleep':'median', 'Exercise_Hours_per_Week':'median', 'Alcohol_Consumption_per_Week':'median'}
               ).astype(int).round(1)

Age_Group,18 - 28,29 - 39,40 - 49,50 - 59,60 - 69,70+
Alcohol_Consumption_per_Week,6,6,5,5,4,4
Calories_Intake,2490,2166,2419,2307,2370,2378
Daily_Steps,11212,9844,12381,9840,10021,8692
Exercise_Hours_per_Week,4,5,5,4,4,4
Hours_of_Sleep,7,7,6,7,6,6


In [ ]:
pd.pivot_table(df_fem, values ='ID', index='Age_Group', columns=('Diabetic'), aggfunc='count')

In [ ]:
pd.pivot_table(df_fem, values ='ID', index='Age_Group', columns=('Heart_Disease'), aggfunc='count')

### Daily steps e exercício por idade e BMI category

In [40]:
pd.pivot_table(df_fem, values= 'Daily_Steps', columns= ('Age_Group'), 
               aggfunc={'Daily_Steps':'median'}).round(-1).astype(int)

Age_Group,18 - 28,29 - 39,40 - 49,50 - 59,60 - 69,70+
Daily_Steps,11210,9840,12380,9840,10020,8690


In [41]:
pd.pivot_table(df_fem, values= 'Exercise_Hours_per_Week', columns= ('Age_Group'), 
               aggfunc={'Exercise_Hours_per_Week':'median'}).round(1)

Age_Group,18 - 28,29 - 39,40 - 49,50 - 59,60 - 69,70+
Exercise_Hours_per_Week,4.6,5.6,5.0,4.9,4.9,4.9


In [42]:
pd.pivot_table(df_fem, values= 'Exercise_Hours_per_Week', columns= ('BMI_Category'), 
               aggfunc={'Exercise_Hours_per_Week':'median'}). round(1)

BMI_Category,Normal,Obese,Overweight
Exercise_Hours_per_Week,5.0,4.9,5.1


In [43]:
pd.pivot_table(df_fem, values= 'Daily_Steps', columns= ('BMI_Category'), 
               aggfunc={'Daily_Steps':'median'}). round(1)

BMI_Category,Normal,Obese,Overweight
Daily_Steps,10458.0,10611.5,8954.0


In [85]:
# Fumante por grupo 
associativa_4 = pd.crosstab(
    index=[df_fem['Age_Group']], 
    columns=df_fem['Smoker']
)
associativa_4

Smoker,No,Yes
Age_Group,,
18 - 28,66,16
29 - 39,60,13
40 - 49,66,16
50 - 59,66,22
60 - 69,58,15
70+,65,14


In [68]:
# BMI por grupo
associativa_5 = pd.crosstab(
    index=[df_fem['BMI_Category']], 
    columns=df_fem['Age_Group']
)
associativa_5


Age_Group,18 - 28,29 - 39,40 - 49,50 - 59,60 - 69,70+
BMI_Category,,,,,,
Normal,25,29,32,33,34,25
Obese,31,27,33,29,17,31
Overweight,26,17,17,26,22,23


In [ ]:
pd.pivot_table(df_fem, values= 'Exercise_Hours_per_Week',index= 'Diabetic', columns= ('BMI_Category'), 
               aggfunc={'Exercise_Hours_per_Week':'median'}). round(1)

BMI_Category,Normal,Obese,Overweight
Diabetic,,,
No,5.3,4.6,4.8
Yes,3.9,6.0,5.2


In [ ]:
pd.pivot_table(df_fem, values= 'Exercise_Hours_per_Week',index= 'Heart_Disease', columns= ('BMI_Category'), 
               aggfunc={'Exercise_Hours_per_Week':'median'}). round(1)

BMI_Category,Normal,Obese,Overweight
Heart_Disease,,,
No,5.1,4.9,5.1
Yes,4.9,4.5,5.9


In [ ]:
pd.pivot_table(df_fem, values= 'Daily_Steps',index= 'Heart_Disease', columns= ('BMI_Category'), 
               aggfunc={'Daily_Steps':'median'}). round(1)

BMI_Category,Normal,Obese,Overweight
Heart_Disease,,,
No,10454.0,10497.0,8954.0
Yes,11361.0,11589.0,8347.0


In [ ]:
# finalizado